<a href="https://colab.research.google.com/github/retuyu88/digitalentkominfo/blob/master/Copy_of_%5B30_1%5D_Layer_wise_Pretraining_ADF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src = "https://i.imgur.com/UjutVJd.jpg" align = "center">

# Perkembangan Deep Learning

Meskipun ditemukan banyak kelemahan, perkembangan untuk memperbaiki Jaringan Saraf Tiruan terus dilakukan hingga mulai populernya istilah ***Deep Learning*** pada tahun 2006 saat Geoff Hinton memperkenalkan teknik baru untuk melatih Jaringan Saraf Tiruan.

# Layer-wise Unsupervised Pretraining

Pada tahun 2006, Geoffrey Hinton and Ruth Salakhutdinov berhasil melatih Jaringan dengan arsitektur layer lebih dari 10 layer dengan cara melatih layer-per-layer satu per satu menggunakan ***Restricted Boltzmann Machine*** hingga bobot layer mampu merepresentasikan input data, baru kemudian menggabungkan layer-layer yang telah dilatih tersebut menjadi satu kesatuan Jaringan. Setelah tertumpuk, Jaringan dapat dilatih dengan Back-Propagation tanpa khawatir mengalami *Vanishing Gradient Problem*

![RBM](https://image.ibb.co/eHq6Ae/mlenewimage14.png)

Implementasi RBM cukup rumit, bahkan menggunakan Tensorflow. Untuk itu mari kita coba proses yang sama dengan metode yang setara namun lebih mudah, yaitu Autoencoder

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


from tensorflow.keras.models import Sequential
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Activation
from tensorflow.keras import backend as K

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Autoencoder

Autoencoder adalah Jaringan Saraf yang dilatih dengan tujuan untuk mengkopi input menjadi output. Jaringan Autoencoder terdiri dari dua bagian: Encoder dan Decoder. Awalnya, Autoencoder digunakan untuk melakukan reduksi dimensi atau *feature learning* yang untuk beberapa kasus berhasil mereduksi dimensi lebih baik daripada PCA.
![autoencoder](https://blog.keras.io/img/ae/autoencoder_schema.jpg)
Kali ini kita juga akan gunakan dataset Fashion MNIST langsung dari dataset tensorflow. Maka load kembali dataset

Gambar: blog.keras.io 

## Load Fashion MNIST

In [0]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

print(x_train.shape)
print(x_test.shape)

In [0]:
# Define the text labels
fashion_mnist_labels = ["T-shirt/top",  # index 0
                        "Trouser",      # index 1
                        "Pullover",     # index 2 
                        "Dress",        # index 3 
                        "Coat",         # index 4
                        "Sandal",       # index 5
                        "Shirt",        # index 6 
                        "Sneaker",      # index 7 
                        "Bag",          # index 8 
                        "Ankle boot"]   # index 9
num_class = 10

Setelah kita unduh dataset, mari kita coba tampilkan tampilan beberapa

In [0]:
fig, ax = plt.subplots(2,10,figsize=(15,4.5))
fig.subplots_adjust(hspace=0.1, wspace=0.1)
for j in range(0,2):
    for i in range(0, 10):
        ax[j,i].imshow(x_train[i+j*10], cmap='gray')
        ax[j,i].set_title(y_train[i+j*10])
        ax[j,i].axis('off')
plt.show()

Kemudian kita lakukan preprocessing terhadap data untuk meratakan data.

In [0]:
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

n_input = x_train.shape[1]

# First Layer Pretrain AutoEncoder
Untuk membuat suatu jaringan Autoencoder, maka kita gunakan sebuah layer Dense dengan input ukuran data dan output ukuran reduksi dimesi.

Setelah itu kita tumpuk dengan sebuah layer Dense lagi yang menerima input berukuran hasil reduksi dimensi dan output kembali ke ukuran data.

In [0]:
encoding_dim = 500 

input_img = Input(shape=(n_input,))

encoded = Dense(encoding_dim, activation='sigmoid')(input_img)  # layer 1

decoded = Dense(n_input, activation='sigmoid')(encoded)         # layer 2

# satukan model
autoencoder = Model(input_img, decoded)

### Train AutoEncoder
Mari kita coba melatih jaringan dengan 30 epoch

In [0]:

autoencoder.compile(optimizer='rmsprop', loss='binary_crossentropy')
autoencoder.fit(x_train, x_train,
                epochs=30,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

### Test Encoder
Untuk melakukan prediksi, maka mari kita buat model Encoder dan Decoder yang masing-masing merupakan setengah dari model yang sudah dilatih

In [0]:
encoder = Model(input_img, encoded)

In [0]:
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))

Mari kita coba untuk meng-encode data test dan kemudian mencoba merekonstruksi kembali hasil encode tersebut

In [0]:
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

In [0]:
import matplotlib.pyplot as plt

n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28,28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28,28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

# Second Layer Pretrain AutoEncoder
Konsep dari *layer-wise pretrain model* adalah dengan membuat jaringan baru lagi yang menerima input hasil reduksi dimensi, dan dilatih untuk mereduksi dimensi lebih jauh.

Untuk itu kita bangung model autoencoder kedua sebagai berikut

In [0]:
encoding_dim2 = 200

input_img2 = Input(shape=(encoding_dim,))
encoded2 = Dense(encoding_dim2, activation='sigmoid')(input_img2)
decoded2 = Dense(encoding_dim, activation='sigmoid')(encoded2)

autoencoder2 = Model(input_img2, decoded2)
autoencoder2.compile(optimizer='rmsprop', loss='binary_crossentropy')

## Train AutoEncoder
Setelah itu, mari kita latih autoencoder kedua dengan data hasil reduksi dimensi autoencoder pertama

In [0]:
x_train2 = encoder.predict(x_train)
autoencoder2.fit(x_train2, x_train2,
                epochs=40,
                batch_size=256,
                shuffle=True,
                validation_data=(encoded_imgs, encoded_imgs))

## Test Encoder 
Kemudian kita juga bangun model encoder dan decoder dari potongan autoencoder kedua

In [0]:
encoder2 = Model(input_img2, encoded2)

encoded_input2 = Input(shape=(encoding_dim2,))
decoder_layer2 = autoencoder2.layers[-1]
decoder2 = Model(encoded_input2, decoder_layer2(encoded_input2))

Mari kita coba kedua autoencoder kita jika digunakan untuk merekonstruksi data test

In [0]:
encoded_imgs = encoder.predict(x_test)
encoded_imgs2 = encoder2.predict(encoded_imgs)
decoded_imgs2 = decoder2.predict(encoded_imgs2)
decoded_imgs = decoder.predict(decoded_imgs2)


decoded_imgs_1 = decoder.predict(encoded_imgs)

In [0]:
import matplotlib.pyplot as plt

n = 10  # how many image we will display
plt.figure(figsize=(20, 6))
for i in range(n):
    # display original
    ax = plt.subplot(3, n, i + 1)
    plt.imshow(x_test[i].reshape(28,28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display first reconstruction
    ax = plt.subplot(3, n, i + 1 + n)
    plt.imshow(decoded_imgs_1[i].reshape(28,28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    # display second reconstruction
    ax = plt.subplot(3, n, i + 1 + n+n)
    plt.imshow(decoded_imgs[i].reshape(28,28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [0]:
autoencoder.summary()
autoencoder2.summary()

# Train Shallow Net using Pretrain
Sekarang jika kita ingin menggunakan bobot hasil pelatihan kedua autoencoder, maka kita bangun model baru yang menggunakan kedua encoder dari autoencoder yang telah dilatih.

Mari kita lihat bagaimana performanya setelah hanya 20 epoch

In [0]:
inputs2 = Input(shape=(n_input,))
x = encoder(inputs2)
x = encoder2(x)
outputs2 = Dense(num_class, activation='softmax')(x)

pretrain = Model(inputs2, outputs2)
pretrain.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

pretrain.fit(x_train, y_train,
                epochs=20,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, y_test))

# Train Shallow Net from Scratch
Kini, mari kita bandingkan dengan Jaringan biasa, dengan 2 layer juga, namun dengan inisialisasi bobot yang random

In [0]:

inputs = Input(shape=(n_input,))
x = Dense(500, activation='sigmoid')(inputs)
x = Dense(200, activation='sigmoid')(x)
outputs = Dense(num_class, activation='softmax')(x)

shallow = Model(inputs, outputs)
shallow.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

shallow.fit(x_train, y_train,
                epochs=20,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, y_test))

In [0]:
print('shallow summary')
shallow.summary()
print()
print('pretrain summary')
pretrain.summary()

# Shallow Scratch vs Pretrain

In [0]:

y_pretrain = np.argmax(pretrain.predict(x_test),axis=1)
y_shallow = np.argmax(shallow.predict(x_test),axis=1)


print('Layer-wise Pretrain Performance')
print('Confusion Matrix')
print(confusion_matrix(np.argmax(y_test,axis=1), y_pretrain))
print()

print('Accuracy =', accuracy_score(np.argmax(y_test,axis=1), y_pretrain))
print()

print('Shallow Performance')
print('Confusion Matrix')
print(confusion_matrix(np.argmax(y_test,axis=1), y_shallow))
print()

print('Accuracy =', accuracy_score(np.argmax(y_test,axis=1), y_shallow))
print()



Dapat dilihat bahwa jaringan yang sama yang dilatih dengan bobot hasil pretraining dapat menghasilkan akurasi lebih baik hanya dalam 20 epoch

<p>Copyright &copy; 2019 <a href=https://www.linkedin.com/in/andityaarifianto/>ADF</a> </p>